In [1]:
!docker-compose up -d --build

time="2025-12-05T22:08:28+07:00" level=warning msg="d:\\CEDT\\Course\\Data_Science\\project\\docker-compose.yml: the attribute `version` is obsolete, it will be ignored, please remove it to avoid potential confusion"
 redis Pulling 
 kafka Pulling 
 zookeeper Pulling 
 f54714f83ac5 Pulling fs layer 
 1fe5b5b014c2 Pulling fs layer 
 36c12cb044ac Pulling fs layer 
 e1e6a53f9063 Pulling fs layer 
 015862cc789d Pulling fs layer 
 3cded401634e Pulling fs layer 
 6bb510d48a09 Pulling fs layer 
 8c144520e9de Pulling fs layer 
 241ffc663b43 Pulling fs layer 
 bcafa67fbd8f Pulling fs layer 
 c180ee728a3e Pulling fs layer 
 c84a6e003d26 Pulling fs layer 
 362eff98b603 Pulling fs layer 
 793133e9ba6b Pulling fs layer 
 2d35ebdb57d9 Pulling fs layer 
 cffa735aaa17 Pulling fs layer 
 6a229eaa0f8a Pulling fs layer 
 4f4fb700ef54 Pulling fs layer 
 4f4fb700ef54 Already exists 
 f54714f83ac5 Download complete 
 1fe5b5b014c2 Download complete 
 e1e6a53f9063 Download complete 
 015862cc789d Downloading 

In [1]:
# โหดข้อมูลจากไฟล์ CSV เข้า Redis

import pandas as pd
import redis
import json

csv_file_path = 'data/bangkok_traffy.csv' 

redis_key = 'traffy_staging_queue'

r = redis.Redis(host='localhost', port=6379, db=0)

BATCH_SIZE = 10000

try:
    total_rows = 0
    pipe = r.pipeline()

    for chunk in pd.read_csv(csv_file_path, chunksize=BATCH_SIZE, dtype=str):
        
        chunk = chunk.fillna("")
        for row in chunk.itertuples(index=False):
            row_dict = row._asdict()
            json_data = json.dumps(row_dict, ensure_ascii=False)
            pipe.lpush(redis_key, json_data)
        
        pipe.execute()
        
        total_rows += len(chunk)
        print(f" ⏳ ส่งแล้ว {total_rows} แถว...", end='\r')

    print(f"✅ เสร็จสิ้น! ส่งทั้งหมด {total_rows} แถว")

except Exception as e:
    print(f"❌ Error: {e}")


✅ เสร็จสิ้น! ส่งทั้งหมด 787026 แถว


In [3]:
data = r.lrange(redis_key, 0, 5)
for item in data:
    print(item.decode('utf-8'))

{"ticket_id": "2025-E7CTFD", "type": "{จราจร}", "organization": "เขตสวนหลวง,สำนักการจราจรและขนส่ง กรุงเทพมหานคร (สจส. กทม.)", "comment": "จริงๆไม่ใช่เขตสวนหลวงแต่ที่ปักเป็นสวนหลวงเพราะผมพักอาศัยอยู่ที่เขตนี้ สิ่งที่จะพูดถึงเส้นจราจรบนพื้นถนนในทั่วกทม. ไม่มีความเข้มไม่มีการสะท้อนในเวลากลางคืน จะออกกฏมากมายตั้งกล้องเพราะจับและปรับมากมาย แต่เส้นจาง บางมองแทบไม่เห็น ทุกๆเขตในกทม. ย้ำเลยว่าจางทุกๆที่ในกทม.", "photo": "https://storage.googleapis.com/traffy_public_bucket/attachment/2025-01/b90079901764000259df5fc4f23e177899cf915f.jpg", "photo_after": "", "coords": "100.65165,13.73036", "address": "2998 ถ. พัฒนาการ แขวงสวนหลวง เขตสวนหลวง กรุงเทพมหานคร 10250 ประเทศไทย", "subdistrict": "สวนหลวง", "district": "สวนหลวง", "province": "กรุงเทพมหานคร", "timestamp": "2025-01-16 02:53:34.290375+00", "state": "กำลังดำเนินการ", "star": "", "count_reopen": "0", "last_activity": "2025-01-16 02:58:02.568451+00"}
{"ticket_id": "EZCZZU", "type": "{ถนน}", "organization": "เขตจอมทอง", "comment": "ปัญหา: พบโรงกล